In [1]:
#!pip install textattack[tensorflow,optional]
#!pip install -U datasets

In [2]:
import os
import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import tensorflow as tf
import transformers
from transformers import CLIPProcessor, CLIPModel

import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


D:\anaconda3\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._p

In [3]:
from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding, WordSwapWordNet 

from textattack.transformations.composite_transformation import CompositeTransformation

from textattack.attack_recipes.attack_recipe import AttackRecipe


class TFLepsilon(AttackRecipe):

    @staticmethod
    def build(model_wrapper):
        #
        # Swap words with their 50 closest embedding nearest-neighbors.
        # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
        
        ## transformation = CompositeTransformation([WordSwapWordNet(), WordSwapEmbedding(max_candidates=50)])
        transformation = WordSwapEmbedding(max_candidates=50)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
        stopwords = set(
            ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", 
             "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", 
             "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren",
             "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", 
             "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", 
             "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", 
             "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", 
             "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly",
             "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", 
             "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", 
             "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", 
             "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", 
             "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", 
             "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no",
             "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on",
             "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", 
             "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", 
             "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", 
             "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", 
             "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those",
             "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless",
             "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't",
             "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", 
             "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", 
             "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn",
             "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself",
             "yourselves"]
        )
        # fmt: on
        constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        #
        constraints.append(WordEmbeddingDistance(min_cos_sim=0.5))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
        use_constraint = UniversalSentenceEncoder(
            threshold = 0.95,
            metric="angular",
            compare_against_original=False,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)


In [4]:
def runAttack(model_wrapper, n, path, target_ds):
    # used TFLepsilon
    attack = TFLepsilon.build(model_wrapper)
    attack_args = textattack.AttackArgs(num_examples=n, log_to_csv=path, shuffle=True, #parallel=True,
                                        checkpoint_interval=5, checkpoint_dir='checkpoints', 
                                        disable_stdout=True)
    attacker = textattack.Attacker(attack, target_ds, attack_args)
    attacker.attack_dataset()

In [5]:
# classifiers
def adaptation(train_set, test_set):
    lr = LogisticRegression(solver='saga', multi_class='multinomial')
    rf = RandomForestClassifier(max_depth=7)
    knn = KNeighborsClassifier(n_neighbors=7)
    svc = SVC(kernel='linear', probability=True)
    
    trainX = train_set.iloc[:,:-1]
    trainY = train_set.iloc[:,-1]
    testX = test_set.iloc[:,:-1]
    testY = test_set.iloc[:,-1]
    
    for key, clf in {'Log Reg':lr, 'RF':rf, 'KNN':knn, 'SVC':svc}.items():
        clf.fit(trainX, trainY) # fit train data
        pred = clf.predict(testX) # predict test set

        # print accuracy
        print(key, (pred==np.array(testY)).sum()/len(pred))

    return lr, rf, knn, svc

In [6]:
target_ds_mr = textattack.datasets.HuggingFaceDataset('rotten_tomatoes', split='test')

textattack: Loading datasets dataset rotten_tomatoes, split test.


### BERT-based-uncased

In [7]:
# bert-based-uncased
model_path1 = 'bert-base-uncased'

bert_model = transformers.AutoModel.from_pretrained(model_path1)
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path1)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [8]:
bertmrtrain = pd.read_csv('dataset_embeddings/bertmrtrain.txt', header=None)
bertmrtest = pd.read_csv('dataset_embeddings/bertmrtest.txt', header=None)
lr_bert_mr, rf_bert_mr, knn_bert_mr, svc_bert_mr = adaptation(bertmrtrain, bertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.8095684803001876
RF 0.7279549718574109
KNN 0.6801125703564728
SVC 0.8030018761726079


In [9]:
# custom model wrapper for CLIP
class BERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [10]:
bert_model_wrapper_lr_mr = BERTModelWrapper(bert_tokenizer, bert_model, lr_bert_mr)
bert_model_wrapper_rf_mr = BERTModelWrapper(bert_tokenizer, bert_model, rf_bert_mr)
bert_model_wrapper_knn_mr = BERTModelWrapper(bert_tokenizer, bert_model, knn_bert_mr)
bert_model_wrapper_svc_mr = BERTModelWrapper(bert_tokenizer, bert_model, svc_bert_mr)

In [11]:
runAttack(bert_model_wrapper_lr_mr, -1, 'attack_results_new/bertmrattacklr095.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacklr095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 1 / 3 / 1 / 5:   0%|                         | 5/1066 [01:29<5:17:09, 17.94s/it]textattack: Saving checkpoint under "checkpoints\1720089154670.ta.chkpt" at 2024-07-04 18:32:34 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 3 / 2 / 6:   1%|▏                        | 6/1066 [01:29<4:24:36, 14.98s/it]

[Succeeded / Failed / Skipped / Total] 1 / 5 / 4 / 10:   1%|▏                      | 10/1066 [01:34<2:46:29,  9.46s/it]textattack: Saving checkpoint under "checkpoints\1720089159588.ta.chkpt" at 2024-07-04 18:32:39 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 9 / 4 / 15:   1%|▎                      | 15/1066 [01:49<2:07:17,  7.27s/it]textattack: Saving checkpoint under "checkpoints\1720089173995.ta.chkpt" at 2024-07-04 18:32:53 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 321 / 534 / 200 / 1055:  99%|██████████████▊| 1055/1066 [37:58<00:23,  2.16s/it]textattack: Saving checkpoint under "checkpoints\1720091343885.ta.chkpt" at 2024-07-04 19:09:03 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 321 / 538 / 201 / 1060:  99%|██████████████▉| 1060/1066 [38:07<00:12,  2.16s/it]textattack: Saving checkpoint under "checkpoints\1720091352287.ta.chkpt" at 2024-07-04 19:09:12 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 321 / 541 / 203 / 1065: 100%|██████████████▉| 1065/1066 [38:15<00:02,  2.16s/it]textattack: Saving checkpoint under "checkpoints\1720091360780.ta.chkpt" at 2024-07-04 19:09:20 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 322 / 541 / 203 / 1066: 100%|███████████████| 1066/1066 [38:18<00:00,  2.16s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 322    |
| Number of failed attacks:     | 541    |
| Number of skipped attacks:    | 203    |
| Original accuracy:            | 80.96% |
| Accuracy under attack:        | 50.75% |
| Attack success rate:          | 37.31% |
| Average perturbed word %:     | 19.68% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 30.88  |
+-------------------------------+--------+


In [12]:
runAttack(bert_model_wrapper_rf_mr, -1, 'attack_results_new/bertmrattackrf095.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackrf095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 2 / 2 / 1 / 5:   0%|                         | 5/1066 [00:29<1:43:00,  5.83s/it]textattack: Saving checkpoint under "checkpoints\1720091393005.ta.chkpt" at 2024-07-04 19:09:53 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 2 / 2 / 6:   1%|▏                        | 6/1066 [00:29<1:26:19,  4.89s/it]

[Succeeded / Failed / Skipped / Total] 2 / 4 / 4 / 10:   1%|▏                      | 10/1066 [00:34<1:00:09,  3.42s/it]textattack: Saving checkpoint under "checkpoints\1720091398057.ta.chkpt" at 2024-07-04 19:09:58 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 8 / 4 / 15:   1%|▎                      | 15/1066 [00:53<1:02:02,  3.54s/it]textattack: Saving checkpoint under "checkpoints\1720091417012.ta.chkpt" at 2024-07-04 19:10:17 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 289 / 479 / 287 / 1055:  99%|██████████████▊| 1055/1066 [34:54<00:21,  1.98s/it]textattack: Saving checkpoint under "checkpoints\1720093457971.ta.chkpt" at 2024-07-04 19:44:17 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 289 / 484 / 287 / 1060:  99%|██████████████▉| 1060/1066 [35:04<00:11,  1.99s/it]textattack: Saving checkpoint under "checkpoints\1720093468231.ta.chkpt" at 2024-07-04 19:44:28 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 290 / 486 / 289 / 1065: 100%|██████████████▉| 1065/1066 [35:12<00:01,  1.98s/it]textattack: Saving checkpoint under "checkpoints\1720093476492.ta.chkpt" at 2024-07-04 19:44:36 after 1065 attacks.
[Succeeded / Failed / Skipped / Total] 290 / 486 / 289 / 1065: 100%|███████████████| 1066/1066 [35:12<00:00,  1.98s/it]

[Succeeded / Failed / Skipped / Total] 290 / 486 / 290 / 1066: 100%|███████████████| 1066/1066 [35:12<00:00,  1.98s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 290    |
| Number of failed attacks:     | 486    |
| Number of skipped attacks:    | 290    |
| Original accuracy:            | 72.8%  |
| Accuracy under attack:        | 45.59% |
| Attack success rate:          | 37.37% |
| Average perturbed word %:     | 18.11% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 30.36  |
+-------------------------------+--------+


In [13]:
runAttack(bert_model_wrapper_knn_mr, -1, 'attack_results_new/bertmrattackknn095.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackknn095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 1 / 2 / 2 / 5:   0%|                         | 5/1066 [00:32<1:54:35,  6.48s/it]textattack: Saving checkpoint under "checkpoints\1720093509620.ta.chkpt" at 2024-07-04 19:45:09 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 2 / 3 / 6:   1%|▏                        | 6/1066 [00:32<1:35:54,  5.43s/it]

[Succeeded / Failed / Skipped / Total] 2 / 4 / 4 / 10:   1%|▏                      | 10/1066 [00:38<1:07:04,  3.81s/it]textattack: Saving checkpoint under "checkpoints\1720093515330.ta.chkpt" at 2024-07-04 19:45:15 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 7 / 6 / 15:   1%|▎                        | 15/1066 [00:50<58:42,  3.35s/it]textattack: Saving checkpoint under "checkpoints\1720093527508.ta.chkpt" at 2024-07-04 19:45:27 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 298 / 420 / 337 / 1055:  99%|██████████████▊| 1055/1066 [29:39<00:18,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1720095256475.ta.chkpt" at 2024-07-04 20:14:16 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 298 / 424 / 338 / 1060:  99%|██████████████▉| 1060/1066 [29:46<00:10,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1720095263874.ta.chkpt" at 2024-07-04 20:14:23 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 298 / 427 / 340 / 1065: 100%|██████████████▉| 1065/1066 [29:55<00:01,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1720095272279.ta.chkpt" at 2024-07-04 20:14:32 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 298 / 427 / 341 / 1066: 100%|███████████████| 1066/1066 [29:55<00:00,  1.68s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 298    |
| Number of failed attacks:     | 427    |
| Number of skipped attacks:    | 341    |
| Original accuracy:            | 68.01% |
| Accuracy under attack:        | 40.06% |
| Attack success rate:          | 41.1%  |
| Average perturbed word %:     | 13.38% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 24.53  |
+-------------------------------+--------+


In [14]:
runAttack(bert_model_wrapper_svc_mr, -1, 'attack_results_new/bertmrattacksvc095.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacksvc095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 1 / 3 / 1 / 5:   0%|                         | 5/1066 [00:23<1:22:58,  4.69s/it]textattack: Saving checkpoint under "checkpoints\1720095296169.ta.chkpt" at 2024-07-04 20:14:56 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 3 / 2 / 6:   1%|▏                        | 6/1066 [00:23<1:09:38,  3.94s/it]

[Succeeded / Failed / Skipped / Total] 1 / 5 / 4 / 10:   1%|▏                        | 10/1066 [00:28<49:56,  2.84s/it]textattack: Saving checkpoint under "checkpoints\1720095301085.ta.chkpt" at 2024-07-04 20:15:01 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 9 / 4 / 15:   1%|▎                        | 15/1066 [00:43<51:00,  2.91s/it]textattack: Saving checkpoint under "checkpoints\1720095316390.ta.chkpt" at 2024-07-04 20:15:16 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 307 / 541 / 207 / 1055:  99%|██████████████▊| 1055/1066 [29:31<00:18,  1.68s/it]textattack: Saving checkpoint under "checkpoints\1720097044217.ta.chkpt" at 2024-07-04 20:44:04 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 307 / 541 / 207 / 1055:  99%|██████████████▊| 1056/1066 [29:31<00:16,  1.68s/it]

[Succeeded / Failed / Skipped / Total] 307 / 545 / 208 / 1060:  99%|██████████████▉| 1060/1066 [29:38<00:10,  1.68s/it]textattack: Saving checkpoint under "checkpoints\1720097051535.ta.chkpt" at 2024-07-04 20:44:11 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 307 / 548 / 210 / 1065: 100%|██████████████▉| 1065/1066 [29:45<00:01,  1.68s/it]textattack: Saving checkpoint under "checkpoints\1720097057773.ta.chkpt" at 2024-07-04 20:44:17 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 308 / 548 / 210 / 1066: 100%|███████████████| 1066/1066 [29:47<00:00,  1.68s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 308    |
| Number of failed attacks:     | 548    |
| Number of skipped attacks:    | 210    |
| Original accuracy:            | 80.3%  |
| Accuracy under attack:        | 51.41% |
| Attack success rate:          | 35.98% |
| Average perturbed word %:     | 20.02% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 30.83  |
+-------------------------------+--------+


### DistilBERT-based-uncased

In [15]:
# bert-based-uncased
model_path2 = 'distilbert/distilbert-base-uncased'

disbert_model = transformers.AutoModel.from_pretrained(model_path2)
disbert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path2)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
disbertmrtrain = pd.read_csv('dataset_embeddings/disbertmrtrain.txt', header=None)
disbertmrtest = pd.read_csv('dataset_embeddings/disbertmrtest.txt', header=None)
lr_disbert_mr, rf_disbert_mr, knn_disbert_mr, svc_disbert_mr = adaptation(disbertmrtrain, disbertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.7945590994371482
RF 0.7682926829268293
KNN 0.7298311444652908
SVC 0.7851782363977486


In [17]:
# custom model wrapper for DistilBERT
class DisBERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).last_hidden_state.mean(axis=1)
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [18]:
disbert_model_wrapper_lr_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, lr_disbert_mr)
disbert_model_wrapper_rf_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, rf_disbert_mr)
disbert_model_wrapper_knn_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, knn_disbert_mr)
disbert_model_wrapper_svc_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, svc_disbert_mr)

In [19]:
runAttack(disbert_model_wrapper_lr_mr, -1, 'attack_results_new/disbertmrattacklr095.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacklr095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 1 / 3 / 1 / 5:   0%|                         | 5/1066 [00:19<1:09:52,  3.95s/it]textattack: Saving checkpoint under "checkpoints\1720097255360.ta.chkpt" at 2024-07-04 20:47:35 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 3 / 3 / 7:   1%|▏                          | 7/1066 [00:19<50:08,  2.84s/it]

[Succeeded / Failed / Skipped / Total] 1 / 6 / 3 / 10:   1%|▏                        | 10/1066 [00:25<44:49,  2.55s/it]textattack: Saving checkpoint under "checkpoints\1720097261068.ta.chkpt" at 2024-07-04 20:47:41 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 9 / 4 / 15:   1%|▎                        | 15/1066 [00:30<36:09,  2.06s/it]textattack: Saving checkpoint under "checkpoints\1720097266573.ta.chkpt" at 2024-07-04 20:47:46 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 300 / 538 / 217 / 1055:  99%|██████████████▊| 1055/1066 [20:53<00:13,  1.19s/it]textattack: Saving checkpoint under "checkpoints\1720098489082.ta.chkpt" at 2024-07-04 21:08:09 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 300 / 538 / 218 / 1056:  99%|██████████████▊| 1056/1066 [20:53<00:11,  1.19s/it]

[Succeeded / Failed / Skipped / Total] 300 / 542 / 218 / 1060:  99%|██████████████▉| 1060/1066 [20:59<00:07,  1.19s/it]textattack: Saving checkpoint under "checkpoints\1720098494609.ta.chkpt" at 2024-07-04 21:08:14 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 300 / 546 / 219 / 1065: 100%|██████████████▉| 1065/1066 [21:07<00:01,  1.19s/it]textattack: Saving checkpoint under "checkpoints\1720098503079.ta.chkpt" at 2024-07-04 21:08:23 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 301 / 546 / 219 / 1066: 100%|███████████████| 1066/1066 [21:08<00:00,  1.19s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 301    |
| Number of failed attacks:     | 546    |
| Number of skipped attacks:    | 219    |
| Original accuracy:            | 79.46% |
| Accuracy under attack:        | 51.22% |
| Attack success rate:          | 35.54% |
| Average perturbed word %:     | 21.65% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 30.92  |
+-------------------------------+--------+


In [20]:
runAttack(disbert_model_wrapper_rf_mr, -1, 'attack_results_new/disbertmrattackrf095.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackrf095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 1 / 3 / 1 / 5:   0%|▏                          | 5/1066 [00:12<42:29,  2.40s/it]textattack: Saving checkpoint under "checkpoints\1720098516150.ta.chkpt" at 2024-07-04 21:08:36 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 3 / 2 / 6:   1%|▏                          | 6/1066 [00:12<35:36,  2.02s/it]

[Succeeded / Failed / Skipped / Total] 2 / 6 / 2 / 10:   1%|▏                        | 10/1066 [00:16<29:47,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1720098521060.ta.chkpt" at 2024-07-04 21:08:41 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 9 / 3 / 15:   1%|▎                        | 15/1066 [00:22<26:27,  1.51s/it]textattack: Saving checkpoint under "checkpoints\1720098526794.ta.chkpt" at 2024-07-04 21:08:46 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 297 / 512 / 246 / 1055:  99%|██████████████▊| 1055/1066 [21:11<00:13,  1.20s/it]textattack: Saving checkpoint under "checkpoints\1720099775311.ta.chkpt" at 2024-07-04 21:29:35 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 297 / 517 / 246 / 1060:  99%|██████████████▉| 1060/1066 [21:18<00:07,  1.21s/it]textattack: Saving checkpoint under "checkpoints\1720099782560.ta.chkpt" at 2024-07-04 21:29:42 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 297 / 521 / 247 / 1065: 100%|██████████████▉| 1065/1066 [21:25<00:01,  1.21s/it]textattack: Saving checkpoint under "checkpoints\1720099789967.ta.chkpt" at 2024-07-04 21:29:49 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 298 / 521 / 247 / 1066: 100%|███████████████| 1066/1066 [21:27<00:00,  1.21s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 298    |
| Number of failed attacks:     | 521    |
| Number of skipped attacks:    | 247    |
| Original accuracy:            | 76.83% |
| Accuracy under attack:        | 48.87% |
| Attack success rate:          | 36.39% |
| Average perturbed word %:     | 21.46% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 33.43  |
+-------------------------------+--------+


In [21]:
runAttack(disbert_model_wrapper_knn_mr, -1, 'attack_results_new/disbertmrattackknn095.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackknn095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 1 / 4 / 0 / 5:   0%|▏                          | 5/1066 [00:12<43:55,  2.48s/it]textattack: Saving checkpoint under "checkpoints\1720099803659.ta.chkpt" at 2024-07-04 21:30:03 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 4 / 1 / 6:   1%|▏                          | 6/1066 [00:12<36:53,  2.09s/it]

[Succeeded / Failed / Skipped / Total] 1 / 7 / 2 / 10:   1%|▏                        | 10/1066 [00:17<30:26,  1.73s/it]textattack: Saving checkpoint under "checkpoints\1720099808539.ta.chkpt" at 2024-07-04 21:30:08 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 9 / 3 / 15:   1%|▎                        | 15/1066 [00:23<27:03,  1.54s/it]textattack: Saving checkpoint under "checkpoints\1720099814404.ta.chkpt" at 2024-07-04 21:30:14 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 279 / 491 / 285 / 1055:  99%|██████████████▊| 1055/1066 [19:52<00:12,  1.13s/it]textattack: Saving checkpoint under "checkpoints\1720100983946.ta.chkpt" at 2024-07-04 21:49:43 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 279 / 495 / 286 / 1060:  99%|██████████████▉| 1060/1066 [19:58<00:06,  1.13s/it]textattack: Saving checkpoint under "checkpoints\1720100989858.ta.chkpt" at 2024-07-04 21:49:49 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 279 / 498 / 288 / 1065: 100%|██████████████▉| 1065/1066 [20:04<00:01,  1.13s/it]textattack: Saving checkpoint under "checkpoints\1720100995317.ta.chkpt" at 2024-07-04 21:49:55 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 280 / 498 / 288 / 1066: 100%|███████████████| 1066/1066 [20:04<00:00,  1.13s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 280    |
| Number of failed attacks:     | 498    |
| Number of skipped attacks:    | 288    |
| Original accuracy:            | 72.98% |
| Accuracy under attack:        | 46.72% |
| Attack success rate:          | 35.99% |
| Average perturbed word %:     | 16.77% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 29.71  |
+-------------------------------+--------+


In [22]:
runAttack(disbert_model_wrapper_svc_mr, -1, 'attack_results_new/disbertmrattacksvc095.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacksvc095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 1 / 3 / 1 / 5:   0%|▏                          | 5/1066 [00:11<41:24,  2.34s/it]textattack: Saving checkpoint under "checkpoints\1720101007572.ta.chkpt" at 2024-07-04 21:50:07 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 3 / 2 / 6:   1%|▏                          | 6/1066 [00:11<34:57,  1.98s/it]

[Succeeded / Failed / Skipped / Total] 1 / 6 / 3 / 10:   1%|▏                        | 10/1066 [00:16<28:57,  1.65s/it]textattack: Saving checkpoint under "checkpoints\1720101012318.ta.chkpt" at 2024-07-04 21:50:12 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 9 / 4 / 15:   1%|▎                        | 15/1066 [00:22<26:02,  1.49s/it]textattack: Saving checkpoint under "checkpoints\1720101018173.ta.chkpt" at 2024-07-04 21:50:18 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 294 / 535 / 226 / 1055:  99%|██████████████▊| 1055/1066 [21:56<00:13,  1.25s/it]textattack: Saving checkpoint under "checkpoints\1720102312285.ta.chkpt" at 2024-07-04 22:11:52 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 294 / 535 / 227 / 1056:  99%|██████████████▊| 1056/1066 [21:56<00:12,  1.25s/it]

[Succeeded / Failed / Skipped / Total] 294 / 539 / 227 / 1060:  99%|██████████████▉| 1060/1066 [22:02<00:07,  1.25s/it]textattack: Saving checkpoint under "checkpoints\1720102318237.ta.chkpt" at 2024-07-04 22:11:58 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 294 / 543 / 228 / 1065: 100%|██████████████▉| 1065/1066 [22:10<00:01,  1.25s/it]textattack: Saving checkpoint under "checkpoints\1720102326048.ta.chkpt" at 2024-07-04 22:12:06 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 295 / 543 / 228 / 1066: 100%|███████████████| 1066/1066 [22:11<00:00,  1.25s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 295    |
| Number of failed attacks:     | 543    |
| Number of skipped attacks:    | 228    |
| Original accuracy:            | 78.61% |
| Accuracy under attack:        | 50.94% |
| Attack success rate:          | 35.2%  |
| Average perturbed word %:     | 21.64% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 31.0   |
+-------------------------------+--------+


### CLIP

In [23]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Access text encoder
text_encoder = clip_model.text_model

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [24]:
clipmrtrain = pd.read_csv('dataset_embeddings/clipmrtrain.txt', header=None)
clipmrtest = pd.read_csv('dataset_embeddings/clipmrtest.txt', header=None)
lr_clip_mr, rf_clip_mr, knn_clip_mr, svc_clip_mr = adaptation(clipmrtrain, clipmrtest)

Log Reg 0.7495309568480301
RF 0.7054409005628518
KNN 0.6885553470919324
SVC 0.7551594746716698


In [25]:
# custom model wrapper for CLIP
class CLIPModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [26]:
clip_model_wrapper_lr_mr = CLIPModelWrapper(clip_processor, text_encoder, lr_clip_mr)
clip_model_wrapper_rf_mr = CLIPModelWrapper(clip_processor, text_encoder, rf_clip_mr)
clip_model_wrapper_knn_mr = CLIPModelWrapper(clip_processor, text_encoder, knn_clip_mr)
clip_model_wrapper_svc_mr = CLIPModelWrapper(clip_processor, text_encoder, svc_clip_mr)

In [27]:
runAttack(clip_model_wrapper_lr_mr, -1, 'attack_results_new/clipmrattacklr095.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacklr095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 2 / 3 / 0 / 5:   0%|▏                          | 5/1066 [00:13<47:46,  2.70s/it]textattack: Saving checkpoint under "checkpoints\1720102768323.ta.chkpt" at 2024-07-04 22:19:28 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 3 / 1 / 6:   1%|▏                          | 6/1066 [00:13<40:16,  2.28s/it]

[Succeeded / Failed / Skipped / Total] 3 / 6 / 1 / 10:   1%|▏                        | 10/1066 [00:18<32:19,  1.84s/it]textattack: Saving checkpoint under "checkpoints\1720102773194.ta.chkpt" at 2024-07-04 22:19:33 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 10 / 1 / 15:   1%|▎                       | 15/1066 [00:29<34:16,  1.96s/it]textattack: Saving checkpoint under "checkpoints\1720102784165.ta.chkpt" at 2024-07-04 22:19:44 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 303 / 487 / 265 / 1055:  99%|██████████████▊| 1055/1066 [20:53<00:13,  1.19s/it]textattack: Saving checkpoint under "checkpoints\1720104008472.ta.chkpt" at 2024-07-04 22:40:08 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 303 / 487 / 266 / 1056:  99%|██████████████▊| 1056/1066 [20:53<00:11,  1.19s/it]

[Succeeded / Failed / Skipped / Total] 303 / 491 / 266 / 1060:  99%|██████████████▉| 1060/1066 [20:59<00:07,  1.19s/it]textattack: Saving checkpoint under "checkpoints\1720104014368.ta.chkpt" at 2024-07-04 22:40:14 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 303 / 495 / 267 / 1065: 100%|██████████████▉| 1065/1066 [21:06<00:01,  1.19s/it]textattack: Saving checkpoint under "checkpoints\1720104021634.ta.chkpt" at 2024-07-04 22:40:21 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 304 / 495 / 267 / 1066: 100%|███████████████| 1066/1066 [21:07<00:00,  1.19s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 304    |
| Number of failed attacks:     | 495    |
| Number of skipped attacks:    | 267    |
| Original accuracy:            | 74.95% |
| Accuracy under attack:        | 46.44% |
| Attack success rate:          | 38.05% |
| Average perturbed word %:     | 18.9%  |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 29.14  |
+-------------------------------+--------+


In [28]:
runAttack(clip_model_wrapper_rf_mr, -1, 'attack_results_new/clipmrattackrf095.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackrf095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 1 / 4 / 0 / 5:   0%|▏                          | 5/1066 [00:12<44:29,  2.52s/it]textattack: Saving checkpoint under "checkpoints\1720104035059.ta.chkpt" at 2024-07-04 22:40:35 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 4 / 1 / 6:   1%|▏                          | 6/1066 [00:12<37:25,  2.12s/it]

[Succeeded / Failed / Skipped / Total] 1 / 7 / 2 / 10:   1%|▏                        | 10/1066 [00:17<30:53,  1.76s/it]textattack: Saving checkpoint under "checkpoints\1720104040030.ta.chkpt" at 2024-07-04 22:40:40 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 10 / 3 / 15:   1%|▎                       | 15/1066 [00:24<29:09,  1.66s/it]textattack: Saving checkpoint under "checkpoints\1720104047441.ta.chkpt" at 2024-07-04 22:40:47 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 247 / 496 / 312 / 1055:  99%|██████████████▊| 1055/1066 [20:03<00:12,  1.14s/it]textattack: Saving checkpoint under "checkpoints\1720105225713.ta.chkpt" at 2024-07-04 23:00:25 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 247 / 496 / 312 / 1055:  99%|██████████████▊| 1056/1066 [20:03<00:11,  1.14s/it]

[Succeeded / Failed / Skipped / Total] 247 / 500 / 313 / 1060:  99%|██████████████▉| 1060/1066 [20:09<00:06,  1.14s/it]textattack: Saving checkpoint under "checkpoints\1720105231706.ta.chkpt" at 2024-07-04 23:00:31 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 247 / 504 / 314 / 1065: 100%|██████████████▉| 1065/1066 [20:16<00:01,  1.14s/it]textattack: Saving checkpoint under "checkpoints\1720105239063.ta.chkpt" at 2024-07-04 23:00:39 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 248 / 504 / 314 / 1066: 100%|███████████████| 1066/1066 [20:17<00:00,  1.14s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 248    |
| Number of failed attacks:     | 504    |
| Number of skipped attacks:    | 314    |
| Original accuracy:            | 70.54% |
| Accuracy under attack:        | 47.28% |
| Attack success rate:          | 32.98% |
| Average perturbed word %:     | 17.91% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 29.66  |
+-------------------------------+--------+


In [29]:
runAttack(clip_model_wrapper_knn_mr, -1, 'attack_results_new/clipmrattackknn095.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackknn095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 1 / 4 / 0 / 5:   0%|▏                          | 5/1066 [00:14<50:25,  2.85s/it]textattack: Saving checkpoint under "checkpoints\1720105253914.ta.chkpt" at 2024-07-04 23:00:53 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 6 / 2 / 10:   1%|▏                        | 10/1066 [00:18<32:07,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1720105257905.ta.chkpt" at 2024-07-04 23:00:57 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 9 / 3 / 15:   1%|▎                        | 15/1066 [00:24<29:08,  1.66s/it]textattack: Saving checkpoint under "checkpoints\1720105264618.ta.chkpt" at 2024-07-04 23:01:04 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 261 / 463 / 331 / 1055:  99%|██████████████▊| 1055/1066 [19:03<00:11,  1.08s/it]textattack: Saving checkpoint under "checkpoints\1720106382800.ta.chkpt" at 2024-07-04 23:19:42 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 261 / 468 / 331 / 1060:  99%|██████████████▉| 1060/1066 [19:10<00:06,  1.09s/it]textattack: Saving checkpoint under "checkpoints\1720106389779.ta.chkpt" at 2024-07-04 23:19:49 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 261 / 472 / 332 / 1065: 100%|██████████████▉| 1065/1066 [19:17<00:01,  1.09s/it]textattack: Saving checkpoint under "checkpoints\1720106397223.ta.chkpt" at 2024-07-04 23:19:57 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 262 / 472 / 332 / 1066: 100%|███████████████| 1066/1066 [19:18<00:00,  1.09s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 262    |
| Number of failed attacks:     | 472    |
| Number of skipped attacks:    | 332    |
| Original accuracy:            | 68.86% |
| Accuracy under attack:        | 44.28% |
| Attack success rate:          | 35.69% |
| Average perturbed word %:     | 15.55% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 27.84  |
+-------------------------------+--------+


In [30]:
runAttack(clip_model_wrapper_svc_mr, -1, 'attack_results_new/clipmrattacksvc095.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacksvc095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 2 / 3 / 0 / 5:   0%|▏                          | 5/1066 [00:12<43:42,  2.47s/it]textattack: Saving checkpoint under "checkpoints\1720106410161.ta.chkpt" at 2024-07-04 23:20:10 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 3 / 1 / 6:   1%|▏                          | 6/1066 [00:12<36:44,  2.08s/it]

[Succeeded / Failed / Skipped / Total] 2 / 6 / 2 / 10:   1%|▏                        | 10/1066 [00:17<30:14,  1.72s/it]textattack: Saving checkpoint under "checkpoints\1720106414985.ta.chkpt" at 2024-07-04 23:20:14 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 9 / 3 / 15:   1%|▎                        | 15/1066 [00:23<27:52,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1720106421685.ta.chkpt" at 2024-07-04 23:20:21 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 304 / 490 / 261 / 1055:  99%|██████████████▊| 1055/1066 [20:43<00:12,  1.18s/it]textattack: Saving checkpoint under "checkpoints\1720107641736.ta.chkpt" at 2024-07-04 23:40:41 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 304 / 494 / 262 / 1060:  99%|██████████████▉| 1060/1066 [20:50<00:07,  1.18s/it]textattack: Saving checkpoint under "checkpoints\1720107647875.ta.chkpt" at 2024-07-04 23:40:47 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 304 / 498 / 263 / 1065: 100%|██████████████▉| 1065/1066 [20:57<00:01,  1.18s/it]textattack: Saving checkpoint under "checkpoints\1720107655642.ta.chkpt" at 2024-07-04 23:40:55 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 305 / 498 / 263 / 1066: 100%|███████████████| 1066/1066 [20:58<00:00,  1.18s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 305    |
| Number of failed attacks:     | 498    |
| Number of skipped attacks:    | 263    |
| Original accuracy:            | 75.33% |
| Accuracy under attack:        | 46.72% |
| Attack success rate:          | 37.98% |
| Average perturbed word %:     | 18.18% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 28.7   |
+-------------------------------+--------+
